In [1]:
from os import listdir
from os.path import isfile, join
folder='SeperatedPatient'
onlyfiles = [f for f in listdir(f'./{folder}') if isfile(join(f'{folder}', f))]
en_folder='EncryptedPatient'

In [2]:
from cryptography.fernet import Fernet
import csv
file_key_dict={}
seen=set()
for patient_id in onlyfiles:
    # Generate a key and instantiate a Fernet object
    key = Fernet.generate_key()
    while key in seen:
        key = Fernet.generate_key()
    seen.add(key)
    file_key_dict[patient_id]=key
    
    cipher_suite = Fernet(key)
    
    # Read the CSV file and encrypt each line
    encrypted_lines = []

    with open(f'./{folder}/{patient_id}', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Convert the row to a string and then to bytes
            row_str = ','.join(row)
            row_bytes = row_str.encode('utf-8')

            # Encrypt the bytes
            encrypted_row = cipher_suite.encrypt(row_bytes)
            encrypted_lines.append(encrypted_row)

    # Write the encrypted data to a new file
    with open(f'./{en_folder}/{patient_id}', 'wb') as file:
        for line in encrypted_lines:
            file.write(line + b'\n')

    
    


In [4]:
# Save dictionary as a CSV file
with open('id_key.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in file_key_dict.items():
        writer.writerow([key, value])
